In [2]:

from diffusers import CogVideoXPipeline, CogVideoXDDIMScheduler, CogVideoXDPMScheduler
import os
import torch

device = "cuda" if torch.cuda.is_available() else "cpu" 

pipe = CogVideoXPipeline.from_pretrained("/mnt/ceph/develop/jiawei/model_checkpoint/CogVideoX-2b-base", torch_dtype=torch.bfloat16).to(device)
 
pipe.load_lora_weights("/mnt/ceph/develop/jiawei/model_checkpoint/export_hf_lora_weights",  weight_name="pytorch_lora_weights.safetensors", adapter_name="test_1")
pipe.fuse_lora(lora_scale=0.7)


pipe.scheduler = CogVideoXDPMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")


Loading pipeline components...: 100%|███████████████████████████████████████████| 5/5 [00:01<00:00,  2.71it/s]


AttributeError: 'CogVideoXPipeline' object has no attribute 'load_lora_weights'

In [3]:

import math 
import random 
import time


os.makedirs("./output", exist_ok=True)

  

In [4]:
prompt="""In the heart of a bustling city, a young woman with long, flowing brown hair and a radiant smile stands out. She's donned in a cozy white beanie adorned with playful animal ears, adding a touch of whimsy to her appearance. Her eyes sparkle with joy as she looks directly into the camera, her expression inviting and warm. The background is a blur of activity, with indistinct figures moving about, suggesting a lively public space. The lighting is soft and diffused, casting a gentle glow on her face and highlighting her features. The overall mood is cheerful and vibrant, capturing a moment of happiness in the midst of urban life.
"""
latents = pipe(
    prompt=prompt,
    num_videos_per_prompt=1,
    num_inference_steps=50,
    num_frames=49,
    use_dynamic_cfg=True,
    output_type="pt",
    guidance_scale=3.0,
    generator=torch.Generator(device="cpu").manual_seed(42),
).frames

100%|█████████████████████████████████████████████████████████████████████████| 50/50 [02:11<00:00,  2.64s/it]


In [8]:

from diffusers.utils import export_to_video
from diffusers.image_processor import VaeImageProcessor
from datetime import datetime, timedelta


batch_size = latents.shape[0]
batch_video_frames = []
for batch_idx in range(batch_size):
    pt_image = latents[batch_idx]
    pt_image = torch.stack([pt_image[i] for i in range(pt_image.shape[0])])

    image_np = VaeImageProcessor.pt_to_numpy(pt_image)
    image_pil = VaeImageProcessor.numpy_to_pil(image_np)
    batch_video_frames.append(image_pil)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
video_path = f"./output/{timestamp}.mp4"
os.makedirs(os.path.dirname(video_path), exist_ok=True)
tensor = batch_video_frames[0]
fps=math.ceil((len(batch_video_frames[0]) - 1) / 6)

export_to_video(tensor, video_path, fps=fps)

'./output/20240911_114547.mp4'